# Tarefa 4

Alunos:

* Andreza - RA: 164213
* Gil - RA: 225323
* Yan - RA: 118982

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [17]:
np.random.seed(42)

In [18]:
df = pd.read_csv("dados4.csv")
X = df.drop(['V15'], axis=1)
y = df.filter(['V15'], axis=1)
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,1,22.08,11.46,2,k,bb,1.585,0,0,0,1,g,100,1213,0
1,0,22.67,7.00,2,c,bb,0.165,0,0,0,0,g,160,1,0
2,0,29.58,1.75,1,k,bb,1.250,0,0,0,1,g,280,1,0
3,0,21.67,11.50,1,j,j,0.000,1,1,11,1,g,0,1,1
4,1,20.17,8.17,2,aa,bb,1.960,1,1,14,0,g,60,159,1


## Preprocessamento

In [19]:
X = pd.get_dummies(df, columns=['V5', 'V6', 'V12'])
X.head(1)

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,...,0,0,0,0,0,0,0,1,0,0


In [20]:
X = StandardScaler().fit_transform(X)
X = pd.DataFrame(X)
X.head(1)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.688737,-0.801052,1.347111,0.54295,-0.190906,-1.047504,-0.864196,-0.493887,1.087908,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079


## Métodos

In [21]:
methods = {
    'Logistic_Regression': LogisticRegression(penalty='none'),
    'Logistic_Regression_l2': LogisticRegression(penalty='l2'),
    'LDA': LinearDiscriminantAnalysis(),
    'QDA': QuadraticDiscriminantAnalysis(),
    'SVM_Linear': LinearSVC(),
    'SVM_RBF': SVC(kernel='rbf'),
    'Naive_Bayes': GaussianNB(),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(),
    'Decision_Tree': DecisionTreeClassifier(),
    'Random_Forest': RandomForestClassifier(),
    'GBM': GradientBoostingClassifier(),
}

params={
    'Logistic_Regression': None,
    'Logistic_Regression_l2': {'alpha': 10**(np.random.uniform(-3, 3, 10))},
    'LDA': None,
    'QDA': None,
    'SVM_Linear': {'C': 2**(np.random.uniform(-5, 15, 10))},
    'SVM_RBF': {'C': 2**(np.random.uniform(-5, 15, 10)),
                'gamma': 2**(np.random.uniform(-9, 3, 10))},
    'KNN': {'n_neighbors': np.random.randint(1, 301, 10)},
    'MLP': {'hidden_layer_sizes': np.arange(5, 21, 3)},
    'Decision_Tree': {'ccp_alpha': np.random.uniform(0.04, 0.0, 10)},
    'Random_Forest': {'n_estimators': [10, 100, 1000],
                      'max_features': [5, 8, 10]},
    'GBM': {'n_estimators':np.random.randint(5, 100, 10),
            'learning_rate': np.random.uniform(0.01, 0.3, 10),
            'max_depth': [2, 3]}
}

## Iterações

In [25]:
results = {
    'clf': [],
    'mean_auc_outer': []#,
#    'best_params': []
}

for method in methods:
    auc_outer = []
    for iteration in range(4):
        X_train, X_test, y_train, y_test = train_test_split(X, np.ravel(y), test_size=0.3)
        if params[method] is not None:
            clf = GridSearchCV(
                    methods[method],
                    params[method],
                    X_train,
                    y_train,
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    refit=True)
        else:
            clf = cross_validate(
                    methods[method],
                    X_train,
                    y_train,
                    cv=StratifiedKFold(n_splits=3),
                    scoring='roc_auc',
                    return_estimator=True)
            clf['test_score']
            index = np.argmax(clf['test_score'])
            clf = clf['estimator'][index]
        # get prediction on test (outer loop)
        y_pred = clf.predict(X_test)
        # calculate auc and save
        auc = roc_auc_score(y_test, y_pred)
        auc_outer.append(auc)
#        break
    mean_auc = np.mean(auc_outer)
    results['clf'].append(method)
    results['mean_auc_outer'].append(mean_auc)
#    results['best_params'].append(clf.best_params_)
#    break


C:\ProgramData\Anaconda3\envs\MO432\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass scoring=           0         1         2         3         4         5         6   \
223  0.688737 -0.167856 -0.001754  0.542950 -0.066806 -1.047504  1.157144   
427  0.688737  0.500798 -0.604823 -1.783978 -0.590118  0.954650 -0.864196   
558  0.688737  3.174573  0.852594  0.542950  0.531264 -1.047504  1.157144   
28  -1.451933 -0.075832  1.455663  0.542950 -0.066806  0.954650  1.157144   
661 -1.451933 -0.765593 -0.873189  0.542950 -0.564700 -1.047504 -0.864196   
..        ...       ...       ...       ...       ...       ...       ...   
108 -1.451933  1.541772  1.774285  0.542950  1.876923  0.954650 -0.864196   
327 -1.451933  1.718223  0.350036  0.542950  0.257647 -1.047504 -0.864196   
57  -1.451933  0.486446 -0.454056  0.542950 -0.602079 -1.047504 -0.864196   
272  0.688737 -1.272149 -0.931485  0.542950 -0.615536 -1.047504 -0.864196   
480 -1.451933 -0.554528  1.522001  0.54

NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [23]:
results_df = pd.DataFrame(results)
results_df

,clf,mean_auc_outer
0,Logistic_Regression,1.0
